In [1]:
import os, sys
import random

import cv2
import numpy as np
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

In [2]:
# register custom dataset
# data acquired here: https://blog.roboflow.com/how-to-train-detectron2/
register_coco_instances(
    'blood_cell_train', 
    {}, 
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/train/_annotations.coco.json",
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/train/"
)

register_coco_instances(
    'blood_cell_val', 
    {}, 
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/val/_annotations.coco.json",
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/val/"
)

register_coco_instances(
    'blood_cell_test', 
    {}, 
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/test/_annotations.coco.json",
    "/home/idealabs/data/opensource_dataset/COCO/bloodcell/test/"
)

In [3]:
# some visualization
blood_cell_train_metadata = MetadataCatalog.get("blood_cell_train")
dataset_dicts = DatasetCatalog.get('blood_cell_train')
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d['file_name'])
    visualizer = Visualizer(img[:, :, ::-1], metadata=blood_cell_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imshow("img", vis.get_image())
    cv2.waitKey(100)

WARNING [12/16 17:21:05 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/16 17:21:05 d2.data.datasets.coco]: Loaded 255 images in COCO format from /home/idealabs/data/opensource_dataset/COCO/bloodcell/train/_annotations.coco.json


In [4]:
# training configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/retinanet_R_50_FPN_1x.yaml'))
cfg.DATASETS.TRAIN = ("blood_cell_train",)
cfg.DATASETS.TEST = ("blood_cell_val",)

cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05

# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.RETINANET.NUM_CLASSES = 4

cfg.TEST.EVAL_PERIOD = 500

Loading config /home/idealabs/Libs/miniconda3/envs/.torchenv/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [5]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [6]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/16 17:21:08 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

WARNING [12/16 17:21:08 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/16 17:21:08 d2.data.datasets.coco]: Loaded 255 images in COCO format from /home/idealabs/data/opensource_dataset/COCO/bloodcell/train/_annotations.coco.json
[12/16 17:21:08 d2.data.build]: Removed 0 images with no usable annotations. 255 images left.
[12/16 17:21:08 d2.data.build]: Distribution of instances among all 4 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|   cells    | 0            | Platelets  | 249          |    RBC     | 2938         |
|    WBC     | 263          |            |              |            |              |
|   total    | 3450         |            |              |            |              |
[12/16 17:21:08 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [Resiz

Skip loading parameter 'head.cls_score.weight' to the model due to incompatible shapes: (720, 256, 3, 3) in the checkpoint but (36, 256, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'head.cls_score.bias' to the model due to incompatible shapes: (720,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.


[12/16 17:21:08 d2.engine.train_loop]: Starting training from iteration 0
[12/16 17:21:15 d2.utils.events]:  eta: 0:07:56  iter: 19  total_loss: 2.507  loss_cls: 1.785  loss_box_reg: 0.6626  time: 0.3361  data_time: 0.0146  lr: 1.9981e-05  max_mem: 2606M
[12/16 17:21:21 d2.utils.events]:  eta: 0:07:41  iter: 39  total_loss: 1.492  loss_cls: 1.13  loss_box_reg: 0.3618  time: 0.3177  data_time: 0.0051  lr: 3.9961e-05  max_mem: 2607M
[12/16 17:21:27 d2.utils.events]:  eta: 0:07:32  iter: 59  total_loss: 1.375  loss_cls: 1.047  loss_box_reg: 0.338  time: 0.3128  data_time: 0.0040  lr: 5.9941e-05  max_mem: 2607M
[12/16 17:21:34 d2.utils.events]:  eta: 0:07:20  iter: 79  total_loss: 1.215  loss_cls: 0.9062  loss_box_reg: 0.3172  time: 0.3105  data_time: 0.0043  lr: 7.9921e-05  max_mem: 2607M
[12/16 17:21:40 d2.utils.events]:  eta: 0:07:14  iter: 99  total_loss: 0.9572  loss_cls: 0.7007  loss_box_reg: 0.2899  time: 0.3079  data_time: 0.0052  lr: 9.9901e-05  max_mem: 2607M
[12/16 17:21:46 d2.u

[12/16 17:23:46 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category   | AP     | category   | AP     | category   | AP     |
|:-----------|:-------|:-----------|:-------|:-----------|:-------|
| cells      | nan    | Platelets  | 43.527 | RBC        | 59.198 |
| WBC        | 75.143 |            |        |            |        |
[12/16 17:23:46 d2.engine.defaults]: Evaluation results for blood_cell_val in csv format:
[12/16 17:23:46 d2.evaluation.testing]: copypaste: Task: bbox
[12/16 17:23:46 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[12/16 17:23:46 d2.evaluation.testing]: copypaste: 59.2892,90.9948,65.8011,42.9318,61.6759,51.1407
[12/16 17:23:46 d2.utils.events]:  eta: 0:05:04  iter: 499  total_loss: 0.385  loss_cls: 0.1861  loss_box_reg: 0.2057  time: 0.3036  data_time: 0.0043  lr: 0.0004995  max_mem: 2607M
[12/16 17:23:52 d2.utils.events]:  eta: 0:04:57  iter: 519  total_loss: 0.3584  loss_cls: 0.1713  loss_box_reg: 0.1901  time: 0.3032  data_time: 0.0

[12/16 17:26:21 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category   | AP     | category   | AP     | category   | AP     |
|:-----------|:-------|:-----------|:-------|:-----------|:-------|
| cells      | nan    | Platelets  | 47.880 | RBC        | 60.948 |
| WBC        | 76.739 |            |        |            |        |
[12/16 17:26:21 d2.engine.defaults]: Evaluation results for blood_cell_val in csv format:
[12/16 17:26:21 d2.evaluation.testing]: copypaste: Task: bbox
[12/16 17:26:21 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[12/16 17:26:21 d2.evaluation.testing]: copypaste: 61.8554,92.0117,70.6537,47.0737,64.6934,49.9441
[12/16 17:26:21 d2.utils.events]:  eta: 0:02:31  iter: 999  total_loss: 0.3206  loss_cls: 0.1316  loss_box_reg: 0.1831  time: 0.3023  data_time: 0.0043  lr: 0.000999  max_mem: 2607M
[12/16 17:26:27 d2.utils.events]:  eta: 0:02:25  iter: 1019  total_loss: 0.2854  loss_cls: 0.1203  loss_box_reg: 0.1673  time: 0.3023  data_time: 0.

[12/16 17:29:01 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category   | AP     | category   | AP     | category   | AP     |
|:-----------|:-------|:-----------|:-------|:-----------|:-------|
| cells      | nan    | Platelets  | 47.939 | RBC        | 60.430 |
| WBC        | 76.771 |            |        |            |        |
[12/16 17:29:01 d2.engine.defaults]: Evaluation results for blood_cell_val in csv format:
[12/16 17:29:01 d2.evaluation.testing]: copypaste: Task: bbox
[12/16 17:29:01 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[12/16 17:29:01 d2.evaluation.testing]: copypaste: 61.7131,91.9649,68.7731,47.3653,63.6909,49.9032
[12/16 17:29:01 d2.utils.events]:  eta: 0:00:00  iter: 1499  total_loss: 0.2983  loss_cls: 0.1272  loss_box_reg: 0.1766  time: 0.3043  data_time: 0.0049  lr: 5e-05  max_mem: 2607M
[12/16 17:29:01 d2.engine.hooks]: Overall training speed: 1498 iterations in 0:07:35 (0.3043 s / it)
[12/16 17:29:01 d2.engine.hooks]: Total training 

In [7]:
#  evaluate
from  detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator('blood_cell_test', cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "blood_cell_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [12/16 17:29:05 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass tasks in directly
WARNING [12/16 17:29:05 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/16 17:29:05 d2.data.datasets.coco]: Loaded 36 images in COCO format from /home/idealabs/data/opensource_dataset/COCO/bloodcell/test/_annotations.coco.json
[12/16 17:29:05 d2.data.build]: Distribution of instances among all 4 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|   cells    | 0            | Platelets  | 36           |    RBC     | 398          |
|    WBC     | 37           |            |              |            |              |
|   total    | 471          |            |              |            |              |
[12/16 17:29:05 d2.data.dataset_ma

OrderedDict([('bbox',
              {'AP': 59.80263815479698,
               'AP50': 89.87663613264635,
               'AP75': 69.3477824837518,
               'APs': 22.951457254049643,
               'APm': 59.97705967256035,
               'APl': 45.24977739143522,
               'AP-cells': nan,
               'AP-Platelets': 42.31911679540153,
               'AP-RBC': 59.60310488849028,
               'AP-WBC': 77.48569278049915})])

In [14]:
# inference on test image
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("blood_cell_test")
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)
blood_cell_test_metadata = MetadataCatalog.get('blood_cell_test')

from detectron2.utils.visualizer import ColorMode
import glob

for image_name in glob.glob('/home/idealabs/data/opensource_dataset/COCO/bloodcell/test/*.jpg'):
    img = cv2.imread(image_name)
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=blood_cell_test_metadata, scale=0.8)
    out = v.draw_instance_predictions(outputs['instances'].to('cpu'))
    cv2.imshow("result", out.get_image()[:, :, ::-1])
    cv2.waitKey(2000)